In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
df = pd.read_csv('train_data.csv')

In [ ]:
X = df.drop("purchaseValue",axis=1)
Y = df["purchaseValue"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=39)

In [ ]:
cat_cols = X_train.select_dtypes(include=['object','bool']).columns.tolist()
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_cols),
    ('num', num_pipeline, num_cols)
])

X_train_tr = preprocessor.fit_transform(X_train)
X_test_tr = preprocessor.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(5, 3))
model.fit(X_train_tr,Y_train)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(Y_train,model.predict(X_train_tr)))

In [ ]:
print(r2_score(Y_test,model.predict(X_test_tr)))

In [ ]:
from sklearn.ensemble import VotingRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

reg = VotingRegressor(
    estimators=[
        ('1',LinearRegression()),
        ('2',GradientBoostingRegressor(random_state=39))
    ]
)

reg.fit(X_train_tr,Y_train)

In [ ]:
print(r2_score(Y_test,reg.predict(X_test_tr)))

In [ ]:
print(r2_score(Y_train,reg.predict(X_train_tr)))

In [ ]:
from sklearn.linear_model import Ridge

reg1 = VotingRegressor(
    estimators=[
        ('1',Ridge(random_state=39)),
        ('2',GradientBoostingRegressor(random_state=39))
    ]
)

reg1.fit(X_train_tr,Y_train)

In [ ]:
print(r2_score(Y_test,reg1.predict(X_test_tr)))
print(r2_score(Y_train,reg1.predict(X_train_tr)))